In [ ]:
from langchain import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.document_loaders import TextLoader, WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
    create_vectorstore_router_agent,
    VectorStoreRouterToolkit,
    VectorStoreInfo,
    create_retriever_tool,
    create_conversational_retrieval_agent
)

## Setting the LLM

In [ ]:
with open("openai_api.txt", "r") as f:
    OPENAI_API = f.read()

llm = OpenAI(
    model_name = "gpt-3.5-turbo-instruct",
    temperature = 0,
    openai_api_key = OPENAI_API
)

The only Toolkit I am going to examine is working with Vectorstores

In [ ]:
## Creating the Client VectorStore

loader = TextLoader("state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key=open("openai_api.txt").read())
client_store = Chroma.from_documents(texts, embeddings, collection_name="client")

In [ ]:
## Creating another VectorStore

loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_store = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")

### One Vectorstore

In [ ]:
## I cannot figure a way to prevent that
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API

In [ ]:
## Initializing Toolkit and Agent

client_vectorstore_info = VectorStoreInfo(
    name = "client info",
    description = "Information about Advengers.",
    vectorstore = client_store
)

toolkit = VectorStoreToolkit(vectorstore_info=client_vectorstore_info)

agent_executor = create_vectorstore_agent(
    llm = llm,
    toolkit = toolkit,
    verbose = True
)

In [ ]:
agent_executor.run("What are Advengers?")

### Multiple Vectorstores

In [ ]:
## Initializing Toolkit and Agent (we could also use `return_direct = True` in this case)

client_vectorstore_info = VectorStoreInfo(
    name = "client info",
    description = "Information about Advengers.",
    vectorstore = client_store
)

ruff_vectorstore_info = VectorStoreInfo(
    name="ruff",
    description="Information about the Ruff python linting library",
    vectorstore=ruff_store
)

router_toolkit = VectorStoreRouterToolkit(
    vectorstores = [client_vectorstore_info, ruff_vectorstore_info],
    llm = llm
)

agent_executor = create_vectorstore_router_agent(
    llm = llm,
    toolkit = router_toolkit,
    verbose = True
)

In [ ]:
agent_executor.run("How are Advengers?")

In [ ]:
agent_executor.run("What tool does ruff use to run over Jupyter Notebooks?")

### Agent with retrieval tool

This is an agent specifically optimized for doing retrieval when necessary and also holding a conversation.

In [ ]:
loader = TextLoader("state_of_the_union.txt")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key=open("openai_api.txt").read())
db = FAISS.from_documents(texts, embeddings)

retriever = db.as_retriever()

In [ ]:
tool = create_retriever_tool(
    retriever,
    "search_state_of_union",
    "Searches and returns documents regarding the state-of-the-union."
)
tools = [tool]

In [ ]:
agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)

In [ ]:
result = agent_executor({"input": "hi, im bob"})

In [ ]:
result = agent_executor({"input": "whats my name?"})

In [ ]:
result = agent_executor({"input": "what did the president say about kentaji brown jackson in the most recent state of the union?"})

In [ ]:
result = agent_executor({"input": "how long ago did he nominate her?"})